In [1]:
!pip install clingo

In [2]:

import clingo
print(clingo.__version__)


5.8.0


In [3]:
import time
import clingo

def solve_asp(program: str = None, files=None, models: int = 0):
    """
    Console-style Clingo output from within a Jupyter notebook.

    Parameters:
    - program:   A string containing an ASP (Answer Set Programming) program (optional).
    - files:     A list of paths to .lp files containing ASP code (optional).
    - models:    Number of models to display (0 = all), similar to the Clingo CLI behavior.

    Example usage:
        solve_asp(program=\"\"\" a. b :- a. #show a/0. \"\"\")
        solve_asp(files=[\"myFirstLp.lp\"], models=0)
    """

    start = time.perf_counter()

    ctl = clingo.Control(arguments=[str(models)] if models is not None else [])
    # (same semantics as CLI: '0' means all models)

    # Add program sources
    if program is not None:
        ctl.add("base", [], program)
    if files:
        for p in files:
            with open(p, "r", encoding="utf-8") as f:
                ctl.add("base", [], f.read())

    # Ground the program
    ctl.ground([("base", [])])

    # Collect and print output models like the CLI
    model_count = 0
    with ctl.solve(yield_=True) as handle:
        for model in handle:
            model_count += 1
            # If there are #show directives, use those; otherwise show all atoms
            shown = model.symbols(shown=True)
            if not shown:
                shown = model.symbols(atoms=True)

            # Sort alphabetically by string representation for stable, neat output
            line = " ".join(sorted(map(str, shown)))
            print(f"Answer: {model_count}")
            if line:
                print(line)

    # Print final result and statistics in Clingo-style format
    status = "SATISFIABLE" if model_count > 0 else "UNSATISFIABLE"
    elapsed = time.perf_counter() - start
    # Minimal statistics, similar to the CLI
    print(status)
    print(f"Models      : {model_count}")
    print(f"Calls       : 1")
    print(f"Time        : {elapsed:.3f}s (Solving: {elapsed:.2f}s 1st Model: {'n/a' if model_count==0 else '0.00s'} Unsat: {'n/a' if model_count>0 else f'{elapsed:.2f}s'})")

In [4]:
program = r"""
person(alice; bob).
rain.
happy(X) :- person(X), not sad(X).
sad(alice) :- rain.
:- happy(X), sad(X).
#show happy/1.
#show sad/1.
"""
solve_asp(program=program, models=0)


Answer: 1
happy(bob) sad(alice)
SATISFIABLE
Models      : 1
Calls       : 1
Time        : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: n/a)


In [12]:
program = r"""

#const n=5.

row(1..n).
col(1..n).
square(X,Y) :- row(X), col(Y).

move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 1 = |X1-X2|, 2 = |Y1-Y2|.
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 2 = |X1-X2|, 1 = |Y1-Y2|.

% Guess exactly one successor and one predecessor per square
1 { succ(X,Y,X2,Y2) : move(X,Y,X2,Y2) } 1 :- square(X,Y).
1 { pred(X,Y,X2,Y2) : move(X2,Y2,X,Y) } 1 :- square(X,Y).

% Link pred/succ for consistency
:- succ(X,Y,X2,Y2), not pred(X2,Y2,X,Y).
:- pred(X,Y,X2,Y2), not succ(X2,Y2,X,Y).

% Ensure single cycle covering all squares (reachability from a fixed anchor)
anchor(1,1).  % pick an arbitrary anchor
reach(X2,Y2) :- anchor(X1,Y1), succ(X1,Y1,X2,Y2).
reach(X3,Y3) :- reach(X2,Y2), succ(X2,Y2,X3,Y3).
:- square(X,Y), not reach(X,Y).

#show succ/4.

"""
solve_asp(program=program, models=0)

UNSATISFIABLE
Models      : 0
Calls       : 1
Time        : 0.004s (Solving: 0.00s 1st Model: n/a Unsat: 0.00s)


In [ ]:
program = r"""

square(1..3, 1..3).

startsquare(1,1).

step(1..8).

move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 1 = |X1-X2|, 2 = |Y1-Y2|.
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 2 = |X1-X2|, 1 = |Y1-Y2|.


route(X,Y,1) :- startsquare(X,Y), step(1).
route(X,Y,8) :- square(X,Y), step(8).

route(X2,Y2,Z2) :- route(X1,Y1,Z1), move(X1,Y1,X2,Y2), step(Z1), step(Z2), Z2 = Z1 + 1.





#show route/3.

"""
solve_asp(program=program, models=0)

Answer: 1
route(1,1,1) route(1,1,3) route(1,1,5) route(1,1,7) route(1,1,8) route(1,2,4) route(1,2,6) route(1,2,8) route(1,3,3) route(1,3,5) route(1,3,7) route(1,3,8) route(2,1,4) route(2,1,6) route(2,1,8) route(2,2,8) route(2,3,2) route(2,3,4) route(2,3,6) route(2,3,8) route(3,1,3) route(3,1,5) route(3,1,7) route(3,1,8) route(3,2,2) route(3,2,4) route(3,2,6) route(3,2,8) route(3,3,5) route(3,3,7) route(3,3,8)
SATISFIABLE
Models      : 1
Calls       : 1
Time        : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: n/a)


In [14]:
program = r"""

% Board
square(1..5, 1..5).

% Knight moves
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 1 = |X1-X2|, 2 = |Y1-Y2|.
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 2 = |X1-X2|, 1 = |Y1-Y2|.

% Steps
step(1..5*5).

% Start
route(1,1,1).

% Exactly one square at each step
{ route(X2,Y2,Z2) : route(X1,Y1,Z1), move(X1,Y1,X2,Y2), Z2 = Z1 + 1 } = 1 :- step(Z2), Z2 > 1.

:- route(X,Y,Z1), route(X,Y,Z2), Z1 != Z2.


#show route/3.

"""
solve_asp(program=program, models=0)

Answer: 1
route(1,1,1) route(1,2,10) route(1,3,25) route(1,4,20) route(1,5,3) route(2,1,24) route(2,2,19) route(2,3,2) route(2,4,9) route(2,5,14) route(3,1,11) route(3,2,8) route(3,3,15) route(3,4,4) route(3,5,21) route(4,1,18) route(4,2,23) route(4,3,6) route(4,4,13) route(4,5,16) route(5,1,7) route(5,2,12) route(5,3,17) route(5,4,22) route(5,5,5)
Answer: 2
route(1,1,1) route(1,2,10) route(1,3,15) route(1,4,20) route(1,5,25) route(2,1,16) route(2,2,19) route(2,3,24) route(2,4,9) route(2,5,14) route(3,1,11) route(3,2,2) route(3,3,17) route(3,4,6) route(3,5,21) route(4,1,18) route(4,2,23) route(4,3,4) route(4,4,13) route(4,5,8) route(5,1,3) route(5,2,12) route(5,3,7) route(5,4,22) route(5,5,5)
Answer: 3
route(1,1,1) route(1,2,10) route(1,3,15) route(1,4,20) route(1,5,23) route(2,1,16) route(2,2,5) route(2,3,22) route(2,4,9) route(2,5,14) route(3,1,11) route(3,2,2) route(3,3,19) route(3,4,24) route(3,5,21) route(4,1,6) route(4,2,17) route(4,3,4) route(4,4,13) route(4,5,8) route(5,1,3) ro

In [8]:
program = r"""

% Board
square(1..3, 1..3).

% Knight moves
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 1 = |X1-X2|, 2 = |Y1-Y2|.
move(X1,Y1,X2,Y2) :- square(X1,Y1), square(X2,Y2), 2 = |X1-X2|, 1 = |Y1-Y2|.

% Steps
step(1..8).

% Start
route(1,1,1).

% Exactly one square at each step
1 { route(X,Y,Z) : square(X,Y) } 1 :- step(Z).

% Legal moves between consecutive steps
:- step(Z), Z < 8,
   route(X1,Y1,Z), route(X2,Y2,Z+1),
   not move(X1,Y1,X2,Y2).

#show route/3.

"""
solve_asp(program=program, models=0)

Answer: 1
route(1,1,1) route(1,1,3) route(1,1,5) route(1,3,7) route(2,3,4) route(3,2,2) route(3,2,6) route(3,2,8)
Answer: 2
route(1,1,1) route(1,1,3) route(1,1,5) route(1,3,7) route(2,1,8) route(2,3,4) route(3,2,2) route(3,2,6)
Answer: 3
route(1,1,1) route(1,1,3) route(1,1,5) route(1,1,7) route(2,3,4) route(3,2,2) route(3,2,6) route(3,2,8)
Answer: 4
route(1,1,1) route(1,1,3) route(1,1,5) route(1,1,7) route(2,3,4) route(2,3,8) route(3,2,2) route(3,2,6)
Answer: 5
route(1,1,1) route(1,1,3) route(1,1,5) route(1,2,8) route(2,3,4) route(2,3,6) route(3,1,7) route(3,2,2)
Answer: 6
route(1,1,1) route(1,1,3) route(1,1,5) route(2,3,4) route(2,3,6) route(2,3,8) route(3,1,7) route(3,2,2)
Answer: 7
route(1,1,1) route(1,1,3) route(1,1,5) route(1,1,7) route(2,3,4) route(2,3,6) route(2,3,8) route(3,2,2)
Answer: 8
route(1,1,1) route(1,1,3) route(1,1,5) route(1,1,7) route(2,3,4) route(2,3,6) route(3,2,2) route(3,2,8)
Answer: 9
route(1,1,1) route(1,1,3) route(1,2,8) route(2,3,4) route(2,3,6) route(3,1,5) 